In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [5]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand con

In [6]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=1)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=1)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=1)
                             )

In [ ]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [8]:
frame

array([[[242, 249, 228],
        [242, 249, 228],
        [242, 248, 230],
        ...,
        [ 76,  60,  90],
        [ 72,  60,  93],
        [ 71,  61,  95]],

       [[242, 249, 227],
        [242, 249, 228],
        [242, 248, 230],
        ...,
        [ 74,  60,  87],
        [ 70,  59,  88],
        [ 68,  59,  89]],

       [[242, 250, 227],
        [242, 249, 228],
        [243, 249, 231],
        ...,
        [ 70,  59,  81],
        [ 68,  59,  82],
        [ 66,  59,  83]],

       ...,

       [[ 39,  45,  52],
        [ 39,  44,  52],
        [ 41,  44,  55],
        ...,
        [ 54,  44,  80],
        [ 55,  43,  81],
        [ 54,  42,  81]],

       [[ 40,  41,  51],
        [ 40,  42,  53],
        [ 41,  42,  55],
        ...,
        [ 51,  39,  77],
        [ 54,  41,  79],
        [ 55,  42,  80]],

       [[ 40,  39,  52],
        [ 41,  40,  53],
        [ 40,  41,  55],
        ...,
        [ 46,  34,  73],
        [ 49,  36,  74],
        [ 55,  42,  80]]

In [9]:
results

mediapipe.python.solution_base.SolutionOutputs

In [10]:
draw_landmarks(frame, results)

In [11]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [12]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [13]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [14]:
result_test = extract_keypoints(results)

In [15]:
len(result_test)

1662

In [16]:
np.save('0', result_test)

In [17]:
np.load('0.npy')

array([ 0.68344676,  0.7833901 , -1.14797151, ...,  0.        ,
        0.        ,  0.        ])

In [134]:
import re

def parse_timestamp(timestamp):
    minutes, seconds = map(int, timestamp.split(':'))
    return minutes * 100 + seconds

def extract_times_and_phrases(text):
    pattern = r'(\d+:\d+) - (\d+)\. (.+)'
    matches = re.findall(pattern, text)
    times = []
    phrases = []
    for match in matches:
        timestamp, _, phrase = match
        times.append(parse_timestamp(timestamp))
        phrases.append(phrase)
    return times, phrases

def generate_time_intervals(times):
    intervals = []
    for i in range(len(times) - 1):
        intervals.append((times[i], times[i+1]))
    return intervals


input_text = '''
0:00 - 1. fine
0:04 - 2. finish
0:06 - 3. forget
0:10 - 4. go
0:12 - 5. hello
0:16 - 6. help
0:18 - 7. don't want
0:22 - 8. like
0:25 - 9. more
0:28 - 10. need
0:30 - 11. wrong
0:34 - 12. no
0:37 - 13. not 
0:40 - 14. please
0:44 - 15. right
0:47 - 16. thank-you
0:49 - 17. want
0:52 - 18. yes
0:55 - 19. bathroom
0:58 - 20. car 
1:02 - 21. home 
1:05 - 22. kitchen 
1:08 - 23. living room
1:12 - 24. phone 
1:15 - 25. cheese
1:18 - 26. cook
1:21 - 27. cookies
1:24 - 28. drink
1:27 - 29. eat/food
1:32 - 30. hamburger
1:35 - 31. milk
1:38 - 32. sandwich 
1:41 - 33. I/me
1:44 - 34. my/mine
1:46 - 35. you
1:49 - 36. your
1:52 - 37. boy
1:55 - 38. girl
1:57 - 39. man
2:00 - 40. woman
2:03 - 41. deaf
2:06 - 42. hearing 
2:10 - 43. how 
2:13 - 44. what 
2:16 - 45. when
2:19 - 46. where
2:22 - 47. who
2:25 - 48. why 
2:27 - 49. asl
2:31 - 50. book
2:33 - 51. class
2:36 - 52. know
2:39 - 53. learn 
2:43 - 54. sentence
2:45 - 55. sign 
2:49 - 56. student
2:53 - 57. teacher 
2:56 - 58. afternoon
2:59 - 59. day
3:02 - 60. hour
3:06 - 61. minute
3:08 - 62. month
3:11 - 63. morning 
3:15 - 64. night
3:18 - 65. noon
3:21 - 66. time
3:24 - 67. week
3:27 - 68. year
3:30 - 69. baby
3:33 - 70. brother
3:36 - 71. children
3:39 - 72. family 
3:41 - 73. father
3:43 - 74. grandfather
3:47 - 75. grandmother
3:49 - 76. husband
3:53 - 77. mother
3:55 - 78. sister 
3:59 - 79. wife
4:01 - 80. dog
4:04 - 81. cat 
4:06 - 82. blue
4:09 - 83. green 
4:12 - 84. orange
4:14 - 85. pink
4:17 - 86. purple 
4:20 - 87. red 
4:22 - 88. yellow 
4:25 - 89. so-so
4:28 - 90. good
4:30 - 91. happy
4:33 - 92. sad
4:36 - 93. bad
4:40 - 94. chat
4:43 - 95. signing
4:47 - 96. look, see, watch
4:50 - 97. pay attention
4:53 - 98. not, don't, doesn't 
4:56 - 99. not yet
4:59 - 100. bye'''

times, phrases = extract_times_and_phrases(input_text)
intervals = generate_time_intervals(times)
print("Time intervals:", intervals)
print("Phrases:", phrases)


Time intervals: [(0, 4), (4, 6), (6, 10), (10, 12), (12, 16), (16, 18), (18, 22), (22, 25), (25, 28), (28, 30), (30, 34), (34, 37), (37, 40), (40, 44), (44, 47), (47, 49), (49, 52), (52, 55), (55, 58), (58, 102), (102, 105), (105, 108), (108, 112), (112, 115), (115, 118), (118, 121), (121, 124), (124, 127), (127, 132), (132, 135), (135, 138), (138, 141), (141, 144), (144, 146), (146, 149), (149, 152), (152, 155), (155, 157), (157, 200), (200, 203), (203, 206), (206, 210), (210, 213), (213, 216), (216, 219), (219, 222), (222, 225), (225, 227), (227, 231), (231, 233), (233, 236), (236, 239), (239, 243), (243, 245), (245, 249), (249, 253), (253, 256), (256, 259), (259, 302), (302, 306), (306, 308), (308, 311), (311, 315), (315, 318), (318, 321), (321, 324), (324, 327), (327, 330), (330, 333), (333, 336), (336, 339), (339, 341), (341, 343), (343, 347), (347, 349), (349, 353), (353, 355), (355, 359), (359, 401), (401, 404), (404, 406), (406, 409), (409, 412), (412, 414), (414, 417), (417, 4

to_doli


import os
from moviepy.editor import *
clip = VideoFileClip("New Project - Made with Clipchamp (1).mp4")
video_folder='output_videos'
for i in range (30,100):
    name=phrases[i]+'.mp4'
    start=intervals[i][0]
    end=intervals[i][1]
    subclip=clip.subclip(start,end)
    video_folder_path = os.path.join(video_folder, name)
    os.mkdir(video_folder_path)
    video_path = os.path.join(video_folder_path, name)  # Ensure unique filenames with appropriate extension
    subclip.write_videofile(video_path, codec='libx264') 
clip.close()


import os
import cv2

# Main folder containing subfolders with extracted frames
frames_folder = "frames_output"
numpy_folder="array"
# Loop through subfolders
for subfolder in os.listdir(frames_folder):
    # Construct the subfolder path
    subfolder_path = os.path.join(frames_folder, subfolder)
    folder_name = os.path.basename(subfolder_path)
    result_path= os.path.join(numpy_folder, folder_name)
    os.mkdir(result_path)
    if os.path.isdir(subfolder_path):  # Check if it's a directory
        # Process image files within the subfolder
        for filename in os.listdir(subfolder_path):
            # Construct the image file path
            image_path = os.path.join(subfolder_path, filename)
            try:
                # Load the image
                frame = cv2.imread(image_path)

            # Make detections
                image, results = mediapipe_detection(frame, holistic)

            # Draw styled landmarks (function not shown)
                draw_styled_landmarks(image, results)
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(result_path,filename)
                np.save(npy_path, keypoints)
            except Exception as e:
                print(f"Error processing image '{image_path}': {e}")

print("Frame flipping completed successfully.")

In [55]:
actions = phrases

# Thirty videos worth of data
no_sequences = 7

# Videos are going to be 30 frames in length
sequence_length = 7
# hello
DATA_PATH='MP_data'

to_do=[28,31,32,33,41,42,43,47,52,54,56,62,71,77,80,82,85,86,87,97]
for actio in to_doli:
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH,actio, str(sequence)))
        except:
            pass
len(to_do)

main_folder='output_videos'
output_folder='array'
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.05, min_tracking_confidence=0.05) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in to_doli:
        video_folder=os.path.join(main_folder, action)
        action+='.mp4'
        video_folder=os.path.join(video_folder,action)
        cap = cv2.VideoCapture(video_folder)
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):
               try:
                # Read frame
                ret, frame = cap.read()

                # Check if frame was read successfully
                if not ret:
                    print(f"Error reading frame {frame_num} from video: {video_file}")
                    continue  # Skip to next iteration if frame not read

                # Make detections, draw landmarks, and export keypoints (unchanged)
                image, results = mediapipe_detection(frame, holistic)
                draw_styled_landmarks(image, results)

                # ... (rest of your frame processing and keypoints export logic)

               except Exception as e:
                    print(f"An error occurred while processing frame {frame_num} from video: {i}")
                    print(f"Error: {e}")
                    continue  # Skip to next iteration if any exception occurs
                # NEW Apply wait logic
               if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
               else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)

                # NEW Export keypoints
               keypoints = extract_keypoints(results)
               npy_path = os.path.join('', actions[i], str(sequence), str(frame_num))
               np.save(npy_path, keypoints)
            # Break gracefully
               if cv2.waitKey(10) & 0xFF == ord('q'):
                break          
    cap.release()
    cv2.destroyAllWindows()

In [62]:
cap = cv2.VideoCapture('bye.mp4')
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    # Loop through sequences aka videos
    for sequence in range(no_sequences):
        # Loop through video length aka sequence length
        for frame_num in range(sequence_length):

            # Read feed
            ret, frame = cap.read()

            # Make detections
            image, results = mediapipe_detection(frame, holistic)
    #                 print(results)

            # Draw landmarks
            draw_styled_landmarks(image, results)

            # NEW Apply wait logic
            if frame_num == 0: 
                cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                cv2.putText(image, 'Collecting frames for {} Video Number {}'.format('bye', sequence), (15,12), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                # Show to screen
                cv2.imshow('OpenCV Feed', image)
                cv2.waitKey(2000)
            else: 
                cv2.putText(image, 'Collecting frames for {} Video Number {}'.format('bye', sequence), (15,12), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                # Show to screen
                cv2.imshow('OpenCV Feed', image)

            # NEW Export keypoints
            keypoints = extract_keypoints(results)
            npy_path = os.path.join(DATA_PATH, 'bye', str(sequence), str(frame_num))
            np.save(npy_path, keypoints)

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
                    
    cap.release()
    cv2.destroyAllWindows()

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [135]:
import os
import numpy as np

def get_subfolders_array(folder_path):
  """
  Gets the subfolder names of a folder and stores them in a NumPy array.

  Args:
    folder_path: The path to the folder.

  Returns:
    A NumPy array containing the subfolder names.
  """

  # Get all subfolders
  subfolders = [f.name for f in os.scandir(folder_path) if f.is_dir()]

  # Convert subfolders to a NumPy array
  subfolders_array = np.array(subfolders)

  return subfolders_array

# Example usage
folder_path = "MP_data"  # Replace with your actual folder path
subfolders_array = get_subfolders_array(folder_path)
subfolders_array

array(['afternoon', 'asl', 'baby', 'bad', 'bathroom', 'blue', 'book',
       'boy', 'brother', 'bye', 'chat', 'cheese', 'children', 'class',
       'cook', 'cookies', 'day', 'deaf', 'dog', "don't want", 'drink',
       'eat-food', 'father', 'fine', 'finish', 'forget', 'girl', 'go',
       'good', 'grandfather', 'grandmother', 'hamburger', 'happy',
       'hello', 'help', 'home', 'hour', 'husband', 'kitchen', 'know',
       'like', 'living room', 'look, see, watch', 'man', 'milk', 'minute',
       'month', 'more', 'mother', 'need', 'night', 'no', 'noon', 'not',
       'not yet', 'orange', 'pay attention', 'pink', 'please', 'right',
       'sad', 'sentence', 'signing', 'so-so', 'student', 'thank-you',
       'time', 'want', 'week', 'when', 'where', 'who', 'wife', 'woman',
       'wrong', 'year', 'yes', 'you', 'your'], dtype='<U16')

In [103]:
label_map = {label:num for num, label in enumerate(subfolders_array)}
label_map

{'afternoon': 0,
 'asl': 1,
 'baby': 2,
 'bad': 3,
 'bathroom': 4,
 'blue': 5,
 'book': 6,
 'boy': 7,
 'brother': 8,
 'bye': 9,
 'chat': 10,
 'cheese': 11,
 'children': 12,
 'class': 13,
 'cook': 14,
 'cookies': 15,
 'day': 16,
 'deaf': 17,
 'dog': 18,
 "don't want": 19,
 'drink': 20,
 'eat-food': 21,
 'father': 22,
 'fine': 23,
 'finish': 24,
 'forget': 25,
 'girl': 26,
 'go': 27,
 'good': 28,
 'grandfather': 29,
 'grandmother': 30,
 'hamburger': 31,
 'happy': 32,
 'hello': 33,
 'help': 34,
 'home': 35,
 'hour': 36,
 'husband': 37,
 'kitchen': 38,
 'know': 39,
 'like': 40,
 'living room': 41,
 'look, see, watch': 42,
 'man': 43,
 'milk': 44,
 'minute': 45,
 'month': 46,
 'more': 47,
 'mother': 48,
 'need': 49,
 'night': 50,
 'no': 51,
 'noon': 52,
 'not': 53,
 'not yet': 54,
 'orange': 55,
 'pay attention': 56,
 'pink': 57,
 'please': 58,
 'right': 59,
 'sad': 60,
 'sentence': 61,
 'signing': 62,
 'so-so': 63,
 'student': 64,
 'thank-you': 65,
 'time': 66,
 'want': 67,
 'week': 68,
 '

In [74]:
sequences, labels = [], []
for action in subfolders_array:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [77]:
np.array(sequences).shape

(553, 7, 1662)

In [80]:
labels

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 23,
 23,
 23,
 23,
 23,
 23,
 23,
 24,
 24,
 24,
 24,
 24,
 24,
 24,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 30,
 30,
 30,
 30,


In [78]:
np.array(labels).shape

(553,)

In [79]:
X = np.array(sequences)

In [81]:
X.shape

(553, 7, 1662)

In [82]:
y = to_categorical(labels).astype(int)

In [84]:
y.shape

(553, 79)

In [85]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [86]:
y_test.shape

(28, 79)

In [53]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [87]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [92]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(7,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(subfolders_array.shape[0], activation='softmax'))

In [93]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [94]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
17/17 [==============================] - 5s 23ms/step - loss: 4.3742 - categorical_accuracy: 0.0114
Epoch 2/2000
17/17 [==============================] - 0s 22ms/step - loss: 4.3702 - categorical_accuracy: 0.0057
Epoch 3/2000
17/17 [==============================] - 0s 23ms/step - loss: 4.3696 - categorical_accuracy: 0.0076
Epoch 4/2000
17/17 [==============================] - 0s 22ms/step - loss: 4.3683 - categorical_accuracy: 0.0152
Epoch 5/2000
17/17 [==============================] - 1s 30ms/step - loss: 4.3563 - categorical_accuracy: 0.0076
Epoch 6/2000
17/17 [==============================] - 0s 22ms/step - loss: 4.2847 - categorical_accuracy: 0.0210
Epoch 7/2000
17/17 [==============================] - 0s 23ms/step - loss: 4.2461 - categorical_accuracy: 0.0210
Epoch 8/2000
17/17 [==============================] - 0s 23ms/step - loss: 4.1686 - categorical_accuracy: 0.0267
Epoch 9/2000
17/17 [==============================] - 0s 22ms/step - loss: 4.1217 - categorical_

17/17 [==============================] - 0s 23ms/step - loss: 3.0928 - categorical_accuracy: 0.1295
Epoch 73/2000
17/17 [==============================] - 0s 22ms/step - loss: 3.0934 - categorical_accuracy: 0.1219
Epoch 74/2000
17/17 [==============================] - 0s 22ms/step - loss: 3.2335 - categorical_accuracy: 0.0990
Epoch 75/2000
17/17 [==============================] - 0s 22ms/step - loss: 3.1641 - categorical_accuracy: 0.0971
Epoch 76/2000
17/17 [==============================] - 0s 22ms/step - loss: 3.0843 - categorical_accuracy: 0.1238
Epoch 77/2000
17/17 [==============================] - 0s 21ms/step - loss: 3.0383 - categorical_accuracy: 0.1562
Epoch 78/2000
17/17 [==============================] - 0s 29ms/step - loss: 3.0949 - categorical_accuracy: 0.1276
Epoch 79/2000
17/17 [==============================] - 0s 22ms/step - loss: 3.0355 - categorical_accuracy: 0.1238
Epoch 80/2000
17/17 [==============================] - 0s 22ms/step - loss: 3.1046 - categorical_accur

17/17 [==============================] - 1s 44ms/step - loss: 2.3027 - categorical_accuracy: 0.2705
Epoch 144/2000
17/17 [==============================] - 1s 39ms/step - loss: 2.3214 - categorical_accuracy: 0.2705
Epoch 145/2000
17/17 [==============================] - 1s 30ms/step - loss: 2.4961 - categorical_accuracy: 0.2648
Epoch 146/2000
17/17 [==============================] - 0s 28ms/step - loss: 2.5568 - categorical_accuracy: 0.2095
Epoch 147/2000
17/17 [==============================] - 0s 25ms/step - loss: 2.3844 - categorical_accuracy: 0.2610
Epoch 148/2000
17/17 [==============================] - 0s 28ms/step - loss: 2.2963 - categorical_accuracy: 0.2895
Epoch 149/2000
17/17 [==============================] - 0s 28ms/step - loss: 2.2761 - categorical_accuracy: 0.2838
Epoch 150/2000
17/17 [==============================] - 0s 28ms/step - loss: 2.4643 - categorical_accuracy: 0.2343
Epoch 151/2000
17/17 [==============================] - 0s 26ms/step - loss: 2.3742 - categoric

17/17 [==============================] - 0s 22ms/step - loss: 1.7180 - categorical_accuracy: 0.4343
Epoch 215/2000
17/17 [==============================] - 0s 22ms/step - loss: 1.7466 - categorical_accuracy: 0.4190
Epoch 216/2000
17/17 [==============================] - 0s 22ms/step - loss: 1.8048 - categorical_accuracy: 0.4095
Epoch 217/2000
17/17 [==============================] - 0s 22ms/step - loss: 1.9549 - categorical_accuracy: 0.3657
Epoch 218/2000
17/17 [==============================] - 0s 22ms/step - loss: 1.8792 - categorical_accuracy: 0.3886
Epoch 219/2000
17/17 [==============================] - 0s 22ms/step - loss: 1.7175 - categorical_accuracy: 0.4533
Epoch 220/2000
17/17 [==============================] - 0s 22ms/step - loss: 1.8229 - categorical_accuracy: 0.3829
Epoch 221/2000
17/17 [==============================] - 0s 22ms/step - loss: 1.7711 - categorical_accuracy: 0.4057
Epoch 222/2000
17/17 [==============================] - 0s 28ms/step - loss: 1.7239 - categoric

17/17 [==============================] - 0s 24ms/step - loss: 1.3300 - categorical_accuracy: 0.5257
Epoch 286/2000
17/17 [==============================] - 0s 23ms/step - loss: 1.3810 - categorical_accuracy: 0.5371
Epoch 287/2000
17/17 [==============================] - 0s 24ms/step - loss: 1.5026 - categorical_accuracy: 0.4990
Epoch 288/2000
17/17 [==============================] - 0s 24ms/step - loss: 1.3864 - categorical_accuracy: 0.5086
Epoch 289/2000
17/17 [==============================] - 0s 24ms/step - loss: 1.4129 - categorical_accuracy: 0.5162
Epoch 290/2000
17/17 [==============================] - 0s 23ms/step - loss: 1.4586 - categorical_accuracy: 0.4743
Epoch 291/2000
17/17 [==============================] - 0s 23ms/step - loss: 1.2737 - categorical_accuracy: 0.5467
Epoch 292/2000
17/17 [==============================] - 0s 23ms/step - loss: 1.4583 - categorical_accuracy: 0.5067
Epoch 293/2000
17/17 [==============================] - 0s 27ms/step - loss: 1.3735 - categoric

...

17/17 [==============================] - 0s 24ms/step - loss: 0.1499 - categorical_accuracy: 0.9714
Epoch 1687/2000
17/17 [==============================] - 0s 24ms/step - loss: 0.1466 - categorical_accuracy: 0.9581
Epoch 1688/2000
17/17 [==============================] - 0s 25ms/step - loss: 0.4066 - categorical_accuracy: 0.8648
Epoch 1689/2000
17/17 [==============================] - 0s 24ms/step - loss: 0.3951 - categorical_accuracy: 0.8590
Epoch 1690/2000
17/17 [==============================] - 0s 28ms/step - loss: 0.2978 - categorical_accuracy: 0.8933
Epoch 1691/2000
17/17 [==============================] - 0s 25ms/step - loss: 0.3517 - categorical_accuracy: 0.8895
Epoch 1692/2000
17/17 [==============================] - 0s 24ms/step - loss: 0.2106 - categorical_accuracy: 0.9295
Epoch 1693/2000
17/17 [==============================] - 0s 25ms/step - loss: 0.1562 - categorical_accuracy: 0.9352
Epoch 1694/2000
17/17 [==============================] - 0s 24ms/step - loss: 0.1368 - c

17/17 [==============================] - 0s 24ms/step - loss: 0.0347 - categorical_accuracy: 0.9962
Epoch 1757/2000
17/17 [==============================] - 0s 25ms/step - loss: 0.0434 - categorical_accuracy: 0.9924
Epoch 1758/2000
17/17 [==============================] - 0s 24ms/step - loss: 0.0792 - categorical_accuracy: 0.9733
Epoch 1759/2000
17/17 [==============================] - 0s 25ms/step - loss: 0.1397 - categorical_accuracy: 0.9448
Epoch 1760/2000
17/17 [==============================] - 0s 25ms/step - loss: 0.1393 - categorical_accuracy: 0.9600
Epoch 1761/2000
17/17 [==============================] - 0s 28ms/step - loss: 0.1460 - categorical_accuracy: 0.9524
Epoch 1762/2000
17/17 [==============================] - 0s 24ms/step - loss: 0.3628 - categorical_accuracy: 0.8819
Epoch 1763/2000
17/17 [==============================] - 0s 25ms/step - loss: 0.5158 - categorical_accuracy: 0.8324
Epoch 1764/2000
17/17 [==============================] - 0s 24ms/step - loss: 0.4215 - c

17/17 [==============================] - 0s 26ms/step - loss: 0.6817 - categorical_accuracy: 0.7905
Epoch 1827/2000
17/17 [==============================] - 1s 31ms/step - loss: 0.4145 - categorical_accuracy: 0.8571
Epoch 1828/2000
17/17 [==============================] - 0s 28ms/step - loss: 0.4716 - categorical_accuracy: 0.8381
Epoch 1829/2000
17/17 [==============================] - 0s 26ms/step - loss: 0.5103 - categorical_accuracy: 0.8305
Epoch 1830/2000
17/17 [==============================] - 0s 27ms/step - loss: 0.2158 - categorical_accuracy: 0.9333
Epoch 1831/2000
17/17 [==============================] - 0s 27ms/step - loss: 0.1075 - categorical_accuracy: 0.9752
Epoch 1832/2000
17/17 [==============================] - 0s 26ms/step - loss: 0.1268 - categorical_accuracy: 0.9695
Epoch 1833/2000
17/17 [==============================] - 0s 27ms/step - loss: 0.2125 - categorical_accuracy: 0.9333
Epoch 1834/2000
17/17 [==============================] - 0s 27ms/step - loss: 0.3059 - c

17/17 [==============================] - 1s 29ms/step - loss: 0.1172 - categorical_accuracy: 0.9695
Epoch 1897/2000
17/17 [==============================] - 0s 27ms/step - loss: 0.1024 - categorical_accuracy: 0.9771
Epoch 1898/2000
17/17 [==============================] - 0s 26ms/step - loss: 0.0839 - categorical_accuracy: 0.9810
Epoch 1899/2000
17/17 [==============================] - 0s 26ms/step - loss: 0.0669 - categorical_accuracy: 0.9829
Epoch 1900/2000
17/17 [==============================] - 0s 27ms/step - loss: 0.0415 - categorical_accuracy: 0.9943
Epoch 1901/2000
17/17 [==============================] - 1s 34ms/step - loss: 0.0564 - categorical_accuracy: 0.9905
Epoch 1902/2000
17/17 [==============================] - 0s 29ms/step - loss: 0.0770 - categorical_accuracy: 0.9752
Epoch 1903/2000
17/17 [==============================] - 0s 28ms/step - loss: 0.0517 - categorical_accuracy: 0.9867
Epoch 1904/2000
17/17 [==============================] - 0s 26ms/step - loss: 0.0888 - c

17/17 [==============================] - 0s 26ms/step - loss: 0.0480 - categorical_accuracy: 0.9867
Epoch 1967/2000
17/17 [==============================] - 1s 31ms/step - loss: 0.0335 - categorical_accuracy: 0.9943
Epoch 1968/2000
17/17 [==============================] - 1s 35ms/step - loss: 0.2946 - categorical_accuracy: 0.9124
Epoch 1969/2000
17/17 [==============================] - 1s 38ms/step - loss: 1.5648 - categorical_accuracy: 0.6667
Epoch 1970/2000
17/17 [==============================] - 1s 36ms/step - loss: 0.5627 - categorical_accuracy: 0.8038
Epoch 1971/2000
17/17 [==============================] - 1s 34ms/step - loss: 0.2842 - categorical_accuracy: 0.9276
Epoch 1972/2000
17/17 [==============================] - 1s 29ms/step - loss: 0.1583 - categorical_accuracy: 0.9581
Epoch 1973/2000
17/17 [==============================] - 0s 27ms/step - loss: 0.1150 - categorical_accuracy: 0.9676
Epoch 1974/2000
17/17 [==============================] - 0s 27ms/step - loss: 0.0652 - c

In [96]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 7, 64)             442112    
                                                                 
 lstm_7 (LSTM)               (None, 7, 128)            98816     
                                                                 
 lstm_8 (LSTM)               (None, 64)                49408     
                                                                 
 dense_5 (Dense)             (None, 64)                4160      
                                                                 
 dense_6 (Dense)             (None, 32)                2080      
                                                                 
 dense_7 (Dense)             (None, 79)                2607      
                                                                 
Total params: 599,183
Trainable params: 599,183
Non-tr

In [97]:
res = model.predict(X_test)

1/1 [==============================] - 1s 1s/step


In [101]:
labels[4]

0

In [98]:
actions[np.argmax(res[4])]

'wrong'

In [104]:
model.save('sign_language_prediction.h5')

In [105]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [106]:
yhat = model.predict(X_test)

1/1 [==============================] - 0s 256ms/step


In [107]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [108]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[27,  0],
        [ 1,  0]],

       [[27,  0],
        [ 1,  0]],

       [[27,  1],
        [ 0,  0]],

       [[27,  0],
        [ 0,  1]],

       [[27,  0],
        [ 1,  0]],

       [[27,  0],
        [ 1,  0]],

       [[26,  1],
        [ 1,  0]],

       [[27,  1],
        [ 0,  0]],

       [[26,  2],
        [ 0,  0]],

       [[27,  1],
        [ 0,  0]],

       [[27,  0],
        [ 0,  1]],

       [[27,  1],
        [ 0,  0]],

       [[26,  1],
        [ 1,  0]],

       [[27,  0],
        [ 1,  0]],

       [[27,  0],
        [ 1,  0]],

       [[27,  1],
        [ 0,  0]],

       [[27,  0],
        [ 1,  0]],

       [[27,  0],
        [ 0,  1]],

       [[27,  0],
        [ 1,  0]],

       [[27,  0],
        [ 1,  0]],

       [[25,  1],
        [ 2,  0]],

       [[27,  0],
        [ 0,  1]],

       [[26,  1],
        [ 1,  0]],

       [[27,  1],
        [ 0,  0]],

       [[27,  1],
        [ 0,  0]],

       [[26,  1],
        [ 1,  0]],

       [[27,

In [109]:
accuracy_score(ytrue, yhat)

0.17857142857142858

In [127]:
from scipy import stats
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        color_index = num % len(colors)  # Get index within color list length
        cv2.rectangle(output_frame, (0, 60+num*40), (int(prob*100), 90+num*40), colors[color_index], -1)
    return output_frame

In [136]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 27ms/step
baby
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
man
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
baby
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
brother
